In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
class AnimeRecommendationModel(nn.Module):
    def __init__(self, num_users, num_animes, genres_dim=10, embed_dim=32):
        super(AnimeRecommendationModel, self).__init__()
        
        # 用户和番剧的嵌入层
        self.user_embedding = nn.Embedding(num_users, embed_dim)  # 用户ID的embedding层
        self.anime_embedding = nn.Embedding(num_animes, embed_dim)  # 番剧ID的embedding层
        
        # 用户的其他特征 (如年龄) 全连接层
        self.user_age_fc = nn.Linear(1, 16)  # 将年龄映射到16维
        
        # 番剧的其他特征 (如评分、收藏数、成员数) 全连接层
        self.anime_meta_fc = nn.Linear(3, 16)  # 将番剧评分，收藏数，成员数映射到16维
        
        # 类别特征嵌入
        self.genre_embedding = nn.Embedding(genres_dim, 8)  # 假设有10类番剧类型，每个映射到8维
        
        # 全连接层
        self.fc1 = nn.Linear(embed_dim * 2 + 16 * 2 + 8, 128)  # 拼接后的输入size：用户和番剧嵌入，其他特征
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # 输出层，预测评分
        
    def forward(self, user_id, user_age, anime_id, anime_meta, genre_id):
        # 用户嵌入特征
        user_embed = self.user_embedding(user_id)  # (batch_size, embed_dim)
        user_age_embed = F.relu(self.user_age_fc(user_age))  # (batch_size, 16)
        
        # 番剧嵌入特征
        anime_embed = self.anime_embedding(anime_id)  # (batch_size, embed_dim)
        anime_meta_embed = F.relu(self.anime_meta_fc(anime_meta))  # (batch_size, 16)
        
        # 番剧类型嵌入
        genre_embed = self.genre_embedding(genre_id).mean(dim=1)  # (batch_size, 8), 对多个genre取均值
        
        # 拼接所有特征
        concat_features = torch.cat([user_embed, user_age_embed, anime_embed, anime_meta_embed, genre_embed], dim=1)
        
        # 全连接层
        x = F.relu(self.fc1(concat_features))
        x = F.relu(self.fc2(x))
        output = self.fc3(x)  # 输出评分
        
        return output